In [21]:
import great_expectations as gx
import pandas as pd
import warnings
warnings.filterwarnings("ignore", message="`result_format` configured at the Validator-level*")

# Load the data
df = pd.read_csv("./data/transactions.csv")
df.index += 2
df["timestamp"] = pd.to_datetime(df["timestamp"])


# Create the ephemeral GX context
context = gx.get_context()

# Add a pandas datasource
data_source = context.data_sources.add_pandas(name="pandas")

# Add a dataframe asset
data_asset = data_source.add_dataframe_asset(name="transactions_data")

# Define the batch (entire DataFrame)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="batch_def")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

# Create the expectation suite with a name
suite = gx.core.expectation_suite.ExpectationSuite(name="transactions_suite")

# Get the validator using the suite
validator = context.get_validator(batch=batch, expectation_suite=suite)

# Add expectations
validator.expect_column_values_to_be_between("amount", min_value=0.01, max_value=100000)

# Add expectations for transaction data
validator.expect_column_values_to_not_be_null("transaction_id")
validator.expect_column_values_to_be_unique("transaction_id")

validator.expect_column_values_to_not_be_null("timestamp")

validator.expect_column_values_to_be_between("amount", min_value=0.01)
validator.expect_column_values_to_not_be_null("currency")
validator.expect_column_values_to_match_regex("currency", r"^[A-Z]{3}$")

validator.expect_column_values_to_not_be_null("sender_account")
validator.expect_column_values_to_not_be_null("receiver_account")

validator.expect_column_values_to_not_be_null("sender_country")
validator.expect_column_values_to_not_be_null("receiver_country")

validator.expect_column_values_to_not_be_null("transaction_type")

validator.expect_column_values_to_not_be_null("notes")

# Validate
results = validator.validate()

# "Self-transfers"
same_account = df[df["sender_account"] == df["receiver_account"]]
print(f"Self-transfers: {len(same_account)} transaktioner")
display(same_account)

# Rader med null-värden
rows_with_nulls = df[df.drop(columns=["notes"]).isnull().any(axis=1)]
print(f"Rader med null-värden: {len(rows_with_nulls)}")
display(rows_with_nulls)

# Transaktioner utan notes
missing_notes = df[df["notes"].isnull() | (df["notes"].str.strip() == "")]
print(f"Transaktioner utan notes: {len(missing_notes)}")
display(missing_notes)

# Internationella transaktioner
non_swedish = df[
    (df["sender_country"].str.lower() != "sweden") |
    (df["receiver_country"].str.lower() != "sweden")
]
print(f"Internationella transaktioner: {len(non_swedish)}")
display(non_swedish)

# Upprepade transaktioner mellan samma konto-par
repeated_pairs = df.groupby(["sender_account", "receiver_account"]).size().reset_index(name="count")
suspicious_pairs = repeated_pairs[repeated_pairs["count"] > 2]
print(f"Upprepade transaktioner (över 2): {len(suspicious_pairs)}")
display(suspicious_pairs)

# Många transaktioner med samma belopp (structured)
duplicate_amounts = df.groupby("amount").size().reset_index(name="count")
common_amounts = duplicate_amounts[duplicate_amounts["count"] > 5]
structured_transactions = df[df["amount"].isin(common_amounts["amount"])]
print(f"Structured transactions: {len(structured_transactions)}")
display(structured_transactions)

# "Smurfing":
small_transactions = df[df["amount"] < 1000]
smurfing_counts = small_transactions.groupby("receiver_account")["sender_account"].nunique().reset_index()
smurfing_candidates = smurfing_counts[smurfing_counts["sender_account"] > 5]
smurfing_transactions = small_transactions[small_transactions["receiver_account"].isin(smurfing_candidates["receiver_account"])]
print(f"Smurfing-transaktioner: {len(smurfing_transactions)}")
display(smurfing_transactions)

# Nattliga transaktioner
df["hour"] = df["timestamp"].dt.hour
night_transactions = df[(df["hour"] >= 0) & (df["hour"] <= 5)]
print(f"Nattliga transaktioner (00–05): {len(night_transactions)}")
display(night_transactions)

# Print results
print(results)


Calculating Metrics: 100%|██████████| 41/41 [00:00<00:00, 115.42it/s]


Self-transfers: 12 transaktioner


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes
328,5c3baa6f-289f-481c-95e8-18957ce653a2,2025-04-20 23:07:00,15614.07,SEK,SE8902KWRS27148391651848,SE8902KWRS27148391651848,Sweden,Uppsala,Sweden,Lidköping,incoming,Freelance project payment
629,7ee1c800-65d3-4d86-8077-07c754d8464c,2025-04-29 16:18:00,23421.03,SEK,SE8902FBTR73223116722714,SE8902FBTR73223116722714,Sweden,Ängelholm,Sweden,Kalmar,incoming,Reimbursement for travel
1141,33f27841-93b7-41dc-aec1-ff796d18fd6e,2025-05-02 00:09:00,21287.34,SEK,SE8902GUBN02340962282734,SE8902GUBN02340962282734,Sweden,Nyköping,Sweden,Lidköping,incoming,Gift to family member
3430,83167861-8440-4f9d-a6c7-d5dc24170d8f,2025-05-01 23:08:00,47855.61,SEK,SE8902HCFQ50732586147290,SE8902HCFQ50732586147290,Sweden,Umeå,Sweden,Uddevalla,outgoing,Gift to family member
3473,4f9d8e01-80f8-4a00-966b-cdc8397bddd8,2025-01-01 15:15:00,32433.22,SEK,SE8902TJBO76009734317061,SE8902TJBO76009734317061,Sweden,Helsingborg,Sweden,Trelleborg,outgoing,Subscription fee
3662,c75aae3b-fd11-409c-9961-b9f1cf095bd0,2025-03-22 10:24:00,42640.40,SEK,SE8902JHMA69643058655073,SE8902JHMA69643058655073,Sweden,Västerås,Sweden,Eskilstuna,incoming,Subscription fee
4412,ec6d64bf-7f17-490e-9be7-9cf0bedd7733,2025-02-22 03:12:00,37328.08,SEK,SE8902KWRS27148391651848,SE8902KWRS27148391651848,Sweden,Falun,Sweden,Sundsvall,incoming,Freelance project payment
6528,8a2affc9-a2ee-4f84-af04-cbb726f5fa52,2025-04-18 03:15:00,33174.44,SEK,SE8902KWAF90270853352878,SE8902KWAF90270853352878,Sweden,Kalmar,Sweden,Skövde,outgoing,Consulting fee
8401,438b3d21-008e-4644-9714-f6fcc46648dd,2025-01-04 00:13:00,23491.08,SEK,SE8902IAAP17064707009052,SE8902IAAP17064707009052,Sweden,Gävle,Sweden,Eskilstuna,outgoing,Utility bill
8753,cd104bad-80f1-45c9-9562-45e46340ca84,2025-03-19 23:24:00,11035.39,SEK,SE8902NRAS31362495279847,SE8902NRAS31362495279847,Sweden,Landskrona,Sweden,Piteå,outgoing,Rent transfer


Rader med null-värden: 1990


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes
13,0308a664-8b78-40bd-9281-8623b3d2584a,2025-03-08 00:00:00,10977.56,SEK,SE8902MPBO87772633723937,SE8902ZJVA11817265425330,NaN,Uddevalla,Sweden,Skövde,outgoing,Rent transfer
41,5aa50eb5-2f63-4125-b1be-975be4bed7e7,2025-04-03 19:09:00,42150.23,SEK,SE8902FCSV26977507325534,SE8902ZCTK81497780781136,Sweden,NaN,Sweden,Falun,outgoing,Rent transfer
45,32b8ef63-4f95-4891-a39a-8b67a2ca57d9,2025-05-02 06:25:00,30955.77,SEK,SE8902NMLM36956633220058,SE8902BWKM27720380863718,NaN,Örnsköldsvik,Sweden,Gävle,incoming,Freelance project payment
116,8523e30f-e027-41e0-9cfc-f1108759bc2a,2025-02-17 09:04:00,31582.56,SEK,SE8902CFVV30512683866714,SE8902DJPX89629848207656,Sweden,NaN,Sweden,Borlänge,incoming,Subscription fee
181,0af039ef-fc14-481b-a528-98b1913056fc,2025-02-11 10:49:00,32588.84,SEK,SE8902ZRZI47417483372579,SE8902PBJH90393502972632,Sweden,Sundsvall,Sweden,NaN,outgoing,Rent transfer
...,...,...,...,...,...,...,...,...,...,...,...,...
99799,5e6f6a84-f346-43f7-81fb-df328cb227d5,2025-03-28 11:00:00,97.65,USD,GB07QBYD82184262811585,SE8902MQZJ06643246897705,Isle of Man,North Shannonmouth,NaN,Lidingö,incoming,Subscription fee
99848,ed3ffb0e-8c6d-4214-ad5c-695e75f01307,2025-03-04 17:00:00,174.66,USD,GB48JAEP49048996177610,SE8902SEYF48760878274175,NaN,Port Chelseaburgh,Sweden,Falun,outgoing,Tax refund
99875,32a96bcb-cffd-42b5-b88f-c1496c00c068,2025-04-12 09:00:00,64.63,USD,GB64YXJK01516973915322,SE8902AJRJ35802643947946,NaN,Justinbury,NaN,Kristianstad,outgoing,Freelance project payment
99973,0415bedd-a564-44f5-aaf6-b3573ef196c9,2025-03-10 13:00:00,138.11,USD,GB97AIZM30478408283369,SE8902FSHA42297476809642,Nicaragua,Robertfurt,Sweden,NaN,incoming,Insurance payout


...

Smurfing-transaktioner: 345


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes
1158,0768d874-012e-4e3d-ab57-d7e7abd68613,2025-02-28 11:26:00,628.56,SEK,SE8902ZWKL91581622045409,SE8902ZNHI21783939608864,Sweden,Östersund,Sweden,Ängelholm,incoming,NaN
1700,fe64bd92-b0dc-4b0f-a112-cf9e6ca89824,2025-02-04 01:48:00,154.70,SEK,SE8902NDNE36671961231780,SE8902XXAF74151570423565,Sweden,Linköping,Sweden,Skellefteå,incoming,Payment for invoice #1833
1960,8b4a5e0c-274c-43b0-a611-e973fbef2b1c,2025-03-14 07:15:00,939.61,SEK,SE8902FCSV26977507325534,SE8902MQZJ06643246897705,Sweden,Falun,Sweden,Motala,outgoing,Loan repayment
4608,40e6acbd-d419-42a4-beaa-e5424b7b697e,2025-04-02 12:28:00,149.23,SEK,SE8902UGGW38274485777478,SE8902MQZJ06643246897705,Sweden,Växjö,Sweden,Skellefteå,incoming,Payment for invoice #1096
4883,8c26b18f-d9c9-4d57-97c9-bfb68adce2dd,2025-05-06 13:04:00,550.96,SEK,SE8902ZJEO82962448424927,SE8902XXAF74151570423565,Sweden,Malmö,Sweden,Karlskrona,outgoing,Utility bill
...,...,...,...,...,...,...,...,...,...,...,...,...
99997,434d8f18-6e14-4cce-9d9f-87df435df2ac,2025-01-08 18:00:00,185.75,USD,GB60MTXG64625720338310,SE8902ZJBK03539793529686,Slovenia,South Lindastad,Sweden,Lidingö,outgoing,Tax refund
99998,f9ab95c4-e33b-478b-8aec-95e182c9582c,2025-03-20 10:00:00,178.02,USD,GB80PSZS32551326940313,SE8902ZJBK03539793529686,Chile,Douglasland,Sweden,NaN,incoming,Gift to family member
99999,f440dba2-38fd-44c0-a215-a6480dfa0eb2,2025-05-12 09:00:00,40.05,USD,GB04QBTS95712926778019,SE8902ZJBK03539793529686,Bolivia,Port Albertchester,Sweden,Nyköping,incoming,Subscription fee
100000,9c239619-faff-42bc-bfd8-dcf4225c00be,2025-03-05 15:00:00,133.11,USD,GB07VGVX87678036612717,SE8902ZJBK03539793529686,Cayman Islands,North David,Sweden,Falun,incoming,Mobile recharge


Nattliga transaktioner (00–05): 23576


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,hour
2,62cacc89-95ca-41c0-a06f-d12950c85a37,2025-01-08 03:17:00,1174.47,SEK,SE8902ZCTK81497780781136,SE8902GGWE97497502378359,Sweden,Södertälje,Sweden,Stockholm,outgoing,Tax refund,3
13,0308a664-8b78-40bd-9281-8623b3d2584a,2025-03-08 00:00:00,10977.56,SEK,SE8902MPBO87772633723937,SE8902ZJVA11817265425330,NaN,Uddevalla,Sweden,Skövde,outgoing,Rent transfer,0
16,ccc76011-9857-40a8-b951-b8c90144449a,2025-04-09 05:56:00,31999.11,SEK,SE8902EHQT53687871208531,SE8902GORN68994003453353,Sweden,Kalmar,Sweden,Nyköping,outgoing,Reimbursement for travel,5
18,e0827340-f916-4f38-b268-ed6570416886,2025-02-26 04:23:00,15452.26,SEK,SE8902BUUB70684057188314,SE8902GHSB29020078073922,Sweden,Uddevalla,Sweden,Göteborg,outgoing,Loan repayment,4
19,4fab6c87-e40f-4e77-8dc2-43aa086c946c,2025-03-20 01:11:00,23696.98,SEK,SE8902JTIU30473679174736,SE8902MPWG51569045484865,Sweden,Helsingborg,Sweden,Norrköping,incoming,Tax refund,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94686,a3fbd1b0-8521-4536-aaae-ac362a217550,2025-05-15 01:50:00,20733.90,USD,GB47ZXMR67509533266059,SE8902SICI51731283266081,Philippines,New Cassandra,Sweden,Karlstad,outgoing,Subscription fee,1
94687,399b5fd8-a005-4999-a9c1-12fffb5903e9,2025-01-17 02:20:00,22126.82,SEK,GB26ANKF67932902146724,SE8902ZIPR73751541971020,Sweden,Västerås,Sweden,Skövde,incoming,Loan repayment,2
94693,7fa2f5c5-2013-4811-abc0-60681dbc4ce6,2025-02-22 03:07:00,33285.80,USD,GB25VXLB68975086674821,SE8902OATV02658940061811,Mauritius,Williamville,Sweden,Nyköping,outgoing,Subscription fee,3
94698,cd29ac9e-d25b-4312-8ba4-2f011499e2f6,2025-03-01 05:48:00,36842.42,SEK,SE8902NRAS31362495279847,GB11NTUR17506359534994,Sweden,Östersund,Eritrea,Josephtown,outgoing,NaN,5


{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "pandas-transactions_data",
          "column": "amount",
          "min_value": 0.01
        },
        "meta": {}
      },
      "result": {
        "element_count": 100000,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "pandas-transactions_data",
          "column": "tr